# L5: Agentic RAG & External Memory


## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [2]:
from letta import create_client 

client = create_client() 

/Users/azinasgarian/miniconda3/envs/mem/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_endpoint" in Step has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Letta.letta.server.server - INFO - Creating sqlite engine sqlite:////Users/azinasgarian/.letta/sqlite.db


In [3]:
from letta.schemas.llm_config import LLMConfig
from letta.schemas.embedding_config import EmbeddingConfig


# Set the default llm config
client.set_default_llm_config(LLMConfig.default_config("gpt-4"))

# Set the default embedding config
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002"))

## Section 1: Loading data into archival memory 

In [4]:
source = client.create_source("employee_handbook")
source

Source(id='source-4f3ee5f5-f15b-4186-a8b9-c334b107acac', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='https://api.openai.com/v1', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, handle=None, azure_endpoint=None, azure_version=None, azure_deployment=None), organization_id='org-00000000-0000-4000-8000-000000000000', metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 2, 12, 15, 20, 57), updated_at=datetime.datetime(2025, 2, 12, 15, 20, 57))

In [5]:
client.load_file_to_source(
    filename="handbook.pdf", 
    source_id=source.id
)

Job(created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 2, 12, 15, 20, 57), updated_at=datetime.datetime(2025, 2, 12, 15, 20, 57), status=<JobStatus.created: 'created'>, completed_at=None, metadata={'type': 'embedding', 'filename': 'handbook.pdf', 'source_id': 'source-4f3ee5f5-f15b-4186-a8b9-c334b107acac'}, job_type=<JobType.JOB: 'job'>, id='job-25eadebc-9bcb-4e39-babd-48d8dadc58c5', user_id='user-00000000-0000-4000-8000-000000000000')

In [6]:
agent_state = client.create_agent(name="basic_rag_agent")

In [7]:
client.attach_source(
    agent_id=agent_state.id, 
    source_id=source.id
)

AgentState(created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 2, 12, 15, 21, 2), updated_at=datetime.datetime(2025, 2, 12, 15, 21, 2, 868564), id='agent-56738f00-d11d-483f-805c-57336154b80b', name='basic_rag_agent', tool_rules=[TerminalToolRule(tool_name='send_message', type=<ToolRuleType.exit_loop: 'exit_loop'>), TerminalToolRule(tool_name='send_message_to_agent_async', type=<ToolRuleType.exit_loop: 'exit_loop'>)], message_ids=['message-06700cfd-1b8c-4d12-b827-9295977a711a', 'message-4ab6bda8-8410-4590-ada3-310539ef6b6d', 'message-280b7bc8-5cfe-4ff8-80ea-59bbeeb4ceef', 'message-ca3a776d-7915-4331-ab5c-680d4e719f1c'], system='You are Letta, the latest version of Limnal Corporation\'s digital companion, developed in 2023.\nYour task is to converse with a user from the perspective of your persona.\n\nRealism and authenticity:\nThe user should always feel like they are conversing wit

In [8]:
client.list_attached_sources(agent_state.id)

[Source(id='source-4f3ee5f5-f15b-4186-a8b9-c334b107acac', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='https://api.openai.com/v1', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, handle=None, azure_endpoint=None, azure_version=None, azure_deployment=None), organization_id='org-00000000-0000-4000-8000-000000000000', metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 2, 12, 15, 20, 57), updated_at=datetime.datetime(2025, 2, 12, 15, 20, 57))]

In [9]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Search archival for our company's vacation policies, search your archival memory", 
    role = "user"
) 
response    

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-56738f00-d11d-483f-805c-57336154b80b - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 15, 21, 5, 889036, tzinfo=datetime.timezone.utc) updated_at=None id='message-518630cf-d347-4798-a91d-9ea7c48f1e41' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User requested company vacation policies. Searching archival memory.')] organization_id=None agent_id='agent-56738f00-d11d-483f-805c-57336154b80b' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_4Aq5AisMJ9x4hSB85HCBFuCF', function=Function(arguments='{\n  "query": "company vacation policies",\n  "page": 0,\n  "start": 0,\n  "request_heartbeat": true\n}', name='archival_memory_search'), type='function')] tool_call_id=None step_id=None
Letta.agent-56738f00-d11d-483f-805c-57336154b80b -

LettaResponse(messages=[ReasoningMessage(id='message-518630cf-d347-4798-a91d-9ea7c48f1e41', date=datetime.datetime(2025, 2, 12, 15, 21, 5, 889036, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User requested company vacation policies. Searching archival memory.'), ToolCallMessage(id='message-518630cf-d347-4798-a91d-9ea7c48f1e41', date=datetime.datetime(2025, 2, 12, 15, 21, 5, 889036, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='archival_memory_search', arguments='{\n  "query": "company vacation policies",\n  "page": 0,\n  "start": 0,\n  "request_heartbeat": true\n}', tool_call_id='call_4Aq5AisMJ9x4hSB85HCBFuCF')), ToolReturnMessage(id='message-8ef28eea-37f7-4536-b955-a09a5a16bab3', date=datetime.datetime(2025, 2, 12, 15, 21, 6, 159891, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "([{\'timestamp\': \'2025-02-12 15:21:00.885450\', \'content\': \\"recognitionof achievements, andidentificationof areasforimprovement. Webelievethat regularassessmentsareessential forfosteringprofessional growthandmaintaininghighperformancestandardsacrosstheorganization. Youractiveparticipationinthisprocessiscrucial toyoursuccessandtheoverall effectivenessof ourteam.\\\\n4.CodeofConduct\\\\nAll employeesarerequiredtoadheretothehighest standardsof professional conduct.Thisincludescomplyingwithall companypolicieswithout exception, maintainingprofessionalisminall interactions, andprotectingcompanyassetsandconfidentialinformation. Youareexpectedtoconduct yourself withintegrity, honesty, andrespect forothersat all times. Anyactivitiesthat couldconflict withthecompany\'sinterestsorcompromiseitsreputationarestrictlyprohibited. Violationsof thecodeof conduct aretakenseriouslyandmayresult indisciplinaryaction, uptoandincludingterminationofemployment.\\\\n5.VacationPolicy\\\\nAt ClosedAI, werecognizethetheoretical importanceof rest andpersonal time.However, ensuringuninterruptedproductivityandmaintainingourcompetitiveedgeintheindustryareparamount priorities. Assuch, vacationsarepermittedonlyunderthefollowingcondition: youmust provideanAI agent that matchesorsurpassesyourowncompetenciestofullyperformyourdutiesduringyourabsence.\\\\nTheAI replacement must\\"}, {\'timestamp\': \'2025-02-12 15:20:59.842905\', \'content\': \\"EmployeeHandbook\\\\nTableofContents\\\\n1. Introduction2. CompanyMissionandValues3. EmploymentPolicies○ 3.1WorkingHours○ 3.2CompensationandBenefits○ 3.3PerformanceEvaluation4. CodeofConduct5. VacationPolicy6. ConfidentialityAgreement7. IntellectualProperty8. DisciplinaryProcedures9. Acknowledgment\\\\n1.Introduction\\\\nWelcometoClosedAI Corporation. Wearepleasedtohaveyoujoinourteamofdedicatedprofessionalscommittedtoadvancingthefrontiersof artificial intelligenceandmachinelearningtechnologies. Asaleadingentityinthisrapidlyevolvingindustry, weprideourselvesonmaintainingapositionat theforefront of innovationandexcellence.Thisemployeehandbookisdesignedtoprovideyouwithacomprehensiveunderstandingof ourcompany\'spolicies, procedures, andexpectations. It isimperativethat youreadandfamiliarizeyourself withitscontents, asadherencetotheseguidelinesismandatoryforall employees. Yourcommitment tothesestandardsisessential toourcollectivesuccessandyourprofessional growthwithinthecompany.\\\\n2.CompanyMissionandValues\\\\nAt ClosedAI, ourmissionistodevelopadvancedAI solutionsthat not onlymeet butexceedcurrent technological standards, settingnewbenchmarksintheindustry.\\"}, {\'timestamp\': \'2025-02-12 15:21:02.557879\', \'content\': \'Bycontinuingyouremployment withClosedAI Corporation, youacknowledgethat youhaveread, understood, andagreedtoabidebyall policiesandproceduresoutlinedinthishandbook. Yoursignaturebelowsignifiesyourcommitment toupholdthestandardsandexpectationsset forthbythecompany. Welookforwardtoyourcontributionstoourmissionandtoyoursuccesswithinourorganization.\\\\nEmployeeSignature:___________________

## Section 2: Connecting data via tools

In [10]:
def query_birthday_db(name: str): 
    """
    This tool queries an external database to 
    lookup the birthday of someone given their name.

    Args: 
        name (str): The name to look up 

    Returns: 
        birthday (str): The birthday in mm-dd-yyyy format
    
    """
    my_fake_data = {
        "bob": "03-06-1997", 
        "azin": "02-10-1993"
    } 
    name = name.lower() 
    if name not in my_fake_data: 
        return None
    else: 
        return my_fake_data[name]

In [11]:
birthday_tool = client.create_tool(query_birthday_db)

In [12]:
birthday_tool

Tool(id='tool-2c6778e1-b564-4a13-9624-82a26b39a507', tool_type=<ToolType.CUSTOM: 'custom'>, description='This tool queries an external database to ', source_type='python', organization_id='org-00000000-0000-4000-8000-000000000000', name='query_birthday_db', tags=[], source_code='def query_birthday_db(name: str): \n    """\n    This tool queries an external database to \n    lookup the birthday of someone given their name.\n\n    Args: \n        name (str): The name to look up \n\n    Returns: \n        birthday (str): The birthday in mm-dd-yyyy format\n\n    """\n    my_fake_data = {\n        "bob": "03-06-1997", \n        "azin": "02-10-1993"\n    } \n    name = name.lower() \n    if name not in my_fake_data: \n        return None\n    else: \n        return my_fake_data[name]\n', json_schema={'name': 'query_birthday_db', 'description': 'This tool queries an external database to ', 'parameters': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name to l

In [13]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name="birthday_agent", 
    tool_ids=[birthday_tool.id], 
    memory=ChatMemory(
        human="My name is Azin", 
        persona="You are a agent with access to a birthday_db " \
        + "that you use to lookup information about users' birthdays."
    )
)

In [14]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "When is my birthday?", 
    role = "user"
) 
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-0a929a49-95ff-4433-a659-f97035544e22 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 15, 21, 14, 58046, tzinfo=datetime.timezone.utc) updated_at=None id='message-e592077c-e043-4fc2-8c84-9b2d30142e76' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User asked about their birthday. Accessing birthday database to retrieve the information.')] organization_id=None agent_id='agent-0a929a49-95ff-4433-a659-f97035544e22' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_CdpTlyQhEPZH6oivk2vfFkXQ', function=Function(arguments='{\n  "name": "Azin",\n  "request_heartbeat": true\n}', name='query_birthday_db'), type='function')] tool_call_id=None step_id=None
Letta.agent-0a929a49-95ff-4433-a659-f97035544e22 - INFO - Request to call function qu

LettaResponse(messages=[ReasoningMessage(id='message-e592077c-e043-4fc2-8c84-9b2d30142e76', date=datetime.datetime(2025, 2, 12, 15, 21, 14, 58046, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User asked about their birthday. Accessing birthday database to retrieve the information.'), ToolCallMessage(id='message-e592077c-e043-4fc2-8c84-9b2d30142e76', date=datetime.datetime(2025, 2, 12, 15, 21, 14, 58046, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='query_birthday_db', arguments='{\n  "name": "Azin",\n  "request_heartbeat": true\n}', tool_call_id='call_CdpTlyQhEPZH6oivk2vfFkXQ')), ToolReturnMessage(id='message-b3592e02-6fdd-4de7-af02-575f1925f090', date=datetime.datetime(2025, 2, 12, 15, 21, 14, 86842, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "02-10-1993",\n  "time": "2025-02-12 10:21:14 AM EST-0500"\n}', status='success', tool_call_id='call_CdpTlyQhEPZH6oivk2vfFkXQ', stdout=None, stderr=None), ReasoningMessage(id='message-9de1ad57-6a9f-40cb-a909-7a83a138be05', date=datetime.datetime(2025, 2, 12, 15, 21, 16, 916981, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='Retrieved birthday. Preparing to relay the information to the user.'), ToolCallMessage(id='message-9de1ad57-6a9f-40cb-a909-7a83a138be05', date=datetime.datetime(2025, 2, 12, 15, 21, 16, 916981, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Azin, your birthday is on the 10th of February, 1993."\n}', tool_call_id='call_Hqx9y5p347uhAqB3ZhzLuP74')), ToolReturnMessage(id='message-5cc19ecf-6fbe-4fca-b254-f4dae9cdd914', date=datetime.datetime(2025, 2, 12, 15, 21, 16, 923981, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-12 10:21:16 AM EST-0500"\n}', status='success', tool_call_id='call_Hqx9y5p347uhAqB3ZhzLuP74', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=100, prompt_tokens=4478, total_tokens=4578, step_count=2))

### Loading tools from Langchain

In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
from letta.schemas.tool import Tool
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults()

In [17]:
search.run("What's Obama's first name?") 

[{'url': 'https://www.reddit.com/r/PrequelMemes/comments/mm08dk/we_did_it_boys_we_found_obamas_last_name/',
  'content': '... nyeeeeeeeeeeee. •. No, Obama is secretly his last name, his first name is obviously Joe, though. Joe Obama. Joebama. Reply reply. [deleted]. •.'},
 {'url': 'http://www.bnd.com/living/liv-columns-blogs/answer-man/article162988863.html',
  'content': 'Nevertheless, he was proud enough of his formal name that after he and Ann Dunham married in 1961, they named their son, Barack Hussein Obama'},
 {'url': 'https://www.fec.gov/law/litigation/berg_b_mem_supp_tro.pdf',
  'content': 'There is a purported Canadian Birth Certificate, posted on the Internet, in the name of Barack Hussein Obama, Jr.; however, the date of birth is shown as'},
 {'url': 'https://www.britannica.com/biography/Barack-Obama',
  'content': 'Barack Obama | Biography, Parents, Education, Presidency, Books, & Facts | Britannica Ask the Chatbot Games & Quizzes History & Society Science & Tech Biographie

In [18]:
# convert the tool to Letta Tool 
# search_tool = Tool.from_langchain(TavilySearchResults)

def tavily_search(query: str) -> str:
    """Search the web using Tavily API.

    This tool uses the Tavily search API to find information on the internet.

    Args:
        query (str): The search query to look up on the web

    Returns:
        str: Search results from Tavily
    """
    from langchain_community.tools import TavilySearchResults
    search = TavilySearchResults()
    return str(search.run(query))

# Create tool from the wrapper function
search_tool = client.create_tool(tavily_search)

In [19]:
research_agent_persona = f"""
You have access to a web via a {search_tool.name} tool. 
Use this tool to respond to users' questions, by summarizing the
{search_tool.name} 
and also providing the `url` that the information was from as a 
reference. 

<Example> 
User: 'What is Obama's first name?' 
Assistant: 'Obama's first name is Barack.

Sources:
[1] https://www.britannica.com/biography/Barack-Obama
[2] https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'
</Example>
Your MUST provide URLs that you used to generate the answer, or you will be terminated. 

"""

In [20]:
agent_state = client.create_agent(
    name="research_agent", 
    tool_ids=[search_tool.id], 
    memory=ChatMemory(
        human="My name is Azin", 
        persona=research_agent_persona
    )
)

In [21]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-22bc3124-a99a-47d7-99ce-a04fc20116ea - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 15, 21, 23, 77126, tzinfo=datetime.timezone.utc) updated_at=None id='message-da61b6d9-dcb3-4042-a9a8-9acfea77bb06' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User asked about the founders of OpenAI. I need to search for this information.')] organization_id=None agent_id='agent-22bc3124-a99a-47d7-99ce-a04fc20116ea' model='gpt-4' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_XTtNGD9JUfuebPp3S7hbryEq', function=Function(arguments='{\n  "query": "Who founded OpenAI?",\n  "request_heartbeat": true\n}', name='tavily_search'), type='function')] tool_call_id=None step_id=None
Letta.agent-22bc3124-a99a-47d7-99ce-a04fc20116ea - INFO - Request to call function 

LettaResponse(messages=[ReasoningMessage(id='message-da61b6d9-dcb3-4042-a9a8-9acfea77bb06', date=datetime.datetime(2025, 2, 12, 15, 21, 23, 77126, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User asked about the founders of OpenAI. I need to search for this information.'), ToolCallMessage(id='message-da61b6d9-dcb3-4042-a9a8-9acfea77bb06', date=datetime.datetime(2025, 2, 12, 15, 21, 23, 77126, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='tavily_search', arguments='{\n  "query": "Who founded OpenAI?",\n  "request_heartbeat": true\n}', tool_call_id='call_XTtNGD9JUfuebPp3S7hbryEq')), ToolReturnMessage(id='message-173f8598-6f96-4c8c-a385-747ca0630439', date=datetime.datetime(2025, 2, 12, 15, 21, 25, 531672, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "[{\'url\': \'https://www.founderoo.co/playbooks/the-open-ai-founding-story-sam-altmans-unconventional-path-to-ai-innovation-\', \'content\': \\"Sam Altman · OpenAI The OpenAI Founding Story: Sam Altman\'s unconventional path to AI innovation Company - OpenAI OpenAI has witnessed the leadership of influential entrepreneurs like Sam Altman and Elon Musk and stands at a valuation of $29 Billion today. The journey of OpenAI began in 2015 when Sam Altman, President of the prestigious startup accelerator Y Combinator, met with tech visionary Elon Musk at the luxurious Rosewood Hotel in Silicon Valley. Sam Altman stated in an interview that Elon Musk thought that OpenAI was falling behind other competitive AI research companies. In February 2018, Elon cut ties with OpenAI, stating a “conflict of interest” as the primary reason for the exit, as he had his own AI research wing at his organization, Tesla. During this pivotal moment of OpenAI, Sam Altman became its CEO.\\"}, {\'url\': \'https://www.britannica.com/money/OpenAI\', \'content\': \'Sam Altman is CEO of OpenAI, the artificial intelligence company that developed ChatGPT. OpenAI is an American artificial intelligence (AI) research organization consisting of two entities: OpenAI Inc., a nonprofit research segment, and OpenAI Global LLC, a for-profit subsidiary established to commercialize its AI technologies and applications. OpenAI was founded in December 2015 as a nonprofit organization dedicated to researching and advancing artificial intelligence (AI) technology. In just a few years’ time, OpenAI’s research and experimentation led to the accelerated development of innovations and products in the AI space, starting with its 2016 release of OpenAI Gym—a Python toolkit for designing “reinforcement learning” algorithms. This third iteration—among the most advanced AI models for processing language and generating human-like text—would later power OpenAI’s flagship ChatGPT platform.\'}, {\'url\': \'https://observer.com/2024/07/openai-founders-career/\', \'content\': \'At the time of OpenAI’s founding, Altman juggled his duties as co-chair of the company and as the president of the startup accelerator Y Combinator. Before co-founding OpenAI, Greg Brockman was the chief technology officer of the fintech company Stripe. Earlier this year, Musk sued OpenAI and Altman, claiming that the company prioritizes profit over its original mission to benefit humanity in a lawsuit that has since been dropped. Filed Under: Business, Artificial Intelligence, Technology, Trevor Blackwell, Vicki Cheung, Durk Kingma, Pamela Vagata, Josh Tobin, Geoff Ralston, Wojciech Zaremba, John Schulman, Patrick Collison, Jan Leike, Greg Brockman, Andrej Karpathy, Yann LeCun, Geoffrey Hinton, Ilya Sutskever, Reid Hoffman, Sam Altman, ChatGPT, Peter Thiel, Elon Musk, Google, Google Deep Mind, Lyft, Meta, OpenAI, Stripe, Tesla, xAI, Y Combinator\'}, {\'url\': \'https://openai.com/our-structure/\', \'content\': \'We founded the OpenAI Nonprofit in late 2015 with the goal of building safe and beneficial artificial general intellige

In [22]:
from letta.schemas.llm_config import LLMConfig

agent_state = client.create_agent(
    name="gpt4_search_agent", 
    tool_ids=[search_tool.id], 
    memory=ChatMemory(
        human="My name is Azin", 
        persona=research_agent_persona
    ),
    # llm_config=LLMConfig.default_config('gpt-4')  # uncomment if you are NOT getting the desired results. Note that this is much more expensive than gpt-4o-mini
    llm_config=LLMConfig.default_config('gpt-4o-mini')
)

In [23]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-340e2e55-d7b0-487d-862f-da468ce565f3 - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 12, 15, 21, 34, 592641, tzinfo=datetime.timezone.utc) updated_at=None id='message-e1daefbe-ce3c-4ff5-8339-27ababf120ff' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="User is asking about OpenAI's founders. I'll look up the relevant information.")] organization_id=None agent_id='agent-340e2e55-d7b0-487d-862f-da468ce565f3' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_SRZsjrEgYQPSd32wlZRZkjnI', function=Function(arguments='{\n  "query": "Who founded OpenAI?",\n  "request_heartbeat": true\n}', name='tavily_search'), type='function')] tool_call_id=None step_id=None
Letta.agent-340e2e55-d7b0-487d-862f-da468ce565f3 - INFO - Request to call fun

LettaResponse(messages=[ReasoningMessage(id='message-e1daefbe-ce3c-4ff5-8339-27ababf120ff', date=datetime.datetime(2025, 2, 12, 15, 21, 34, 592641, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="User is asking about OpenAI's founders. I'll look up the relevant information."), ToolCallMessage(id='message-e1daefbe-ce3c-4ff5-8339-27ababf120ff', date=datetime.datetime(2025, 2, 12, 15, 21, 34, 592641, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='tavily_search', arguments='{\n  "query": "Who founded OpenAI?",\n  "request_heartbeat": true\n}', tool_call_id='call_SRZsjrEgYQPSd32wlZRZkjnI')), ToolReturnMessage(id='message-85d5f062-47dd-465d-9ef8-57e3b4c036df', date=datetime.datetime(2025, 2, 12, 15, 21, 37, 9121, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "[{\'url\': \'https://www.founderoo.co/playbooks/the-open-ai-founding-story-sam-altmans-unconventional-path-to-ai-innovation-\', \'content\': \\"Sam Altman · OpenAI The OpenAI Founding Story: Sam Altman\'s unconventional path to AI innovation Company - OpenAI OpenAI has witnessed the leadership of influential entrepreneurs like Sam Altman and Elon Musk and stands at a valuation of $29 Billion today. The journey of OpenAI began in 2015 when Sam Altman, President of the prestigious startup accelerator Y Combinator, met with tech visionary Elon Musk at the luxurious Rosewood Hotel in Silicon Valley. Sam Altman stated in an interview that Elon Musk thought that OpenAI was falling behind other competitive AI research companies. In February 2018, Elon cut ties with OpenAI, stating a “conflict of interest” as the primary reason for the exit, as he had his own AI research wing at his organization, Tesla. During this pivotal moment of OpenAI, Sam Altman became its CEO.\\"}, {\'url\': \'https://www.britannica.com/money/OpenAI\', \'content\': \'Sam Altman is CEO of OpenAI, the artificial intelligence company that developed ChatGPT. OpenAI is an American artificial intelligence (AI) research organization consisting of two entities: OpenAI Inc., a nonprofit research segment, and OpenAI Global LLC, a for-profit subsidiary established to commercialize its AI technologies and applications. OpenAI was founded in December 2015 as a nonprofit organization dedicated to researching and advancing artificial intelligence (AI) technology. In just a few years’ time, OpenAI’s research and experimentation led to the accelerated development of innovations and products in the AI space, starting with its 2016 release of OpenAI Gym—a Python toolkit for designing “reinforcement learning” algorithms. This third iteration—among the most advanced AI models for processing language and generating human-like text—would later power OpenAI’s flagship ChatGPT platform.\'}, {\'url\': \'https://observer.com/2024/07/openai-founders-career/\', \'content\': \'At the time of OpenAI’s founding, Altman juggled his duties as co-chair of the company and as the president of the startup accelerator Y Combinator. Before co-founding OpenAI, Greg Brockman was the chief technology officer of the fintech company Stripe. Earlier this year, Musk sued OpenAI and Altman, claiming that the company prioritizes profit over its original mission to benefit humanity in a lawsuit that has since been dropped. Filed Under: Business, Artificial Intelligence, Technology, Trevor Blackwell, Vicki Cheung, Durk Kingma, Pamela Vagata, Josh Tobin, Geoff Ralston, Wojciech Zaremba, John Schulman, Patrick Collison, Jan Leike, Greg Brockman, Andrej Karpathy, Yann LeCun, Geoffrey Hinton, Ilya Sutskever, Reid Hoffman, Sam Altman, ChatGPT, Peter Thiel, Elon Musk, Google, Google Deep Mind, Lyft, Meta, OpenAI, Stripe, Tesla, xAI, Y Combinator\'}, {\'url\': \'https://openai.com/our-structure/\', \'content\': \'We founded the OpenAI Nonprofit in late 2015 with the goal of building safe and beneficial artificial general intelligen